# Flight Price Prediction

In [1]:
# importing libraries/dependencies

import selenium
import pandas as pd
import numpy as np
import time
import requests
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

# Scraping Flight Ticket price from https://www.yatra.com/

In [2]:
# connecting to the webdriver

driver = webdriver.Chrome(r"C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe")
driver.maximize_window()

In [3]:
# getting the webpage of mentioned url
url = "https://www.yatra.com/"
driver.get(url)
time.sleep(3)

In [4]:
# clicking on search flights button to navigate price of flight tickets
driver.find_element_by_xpath("/html/body/div[2]/div/section/div[1]/div/div[1]/section/div/div/div/div[1]/div[3]/div[2]/input").click()
time.sleep(3)

In [5]:
# let's make a list of locations to scrape data
loc_lst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai', 'Hyderabad', 'Goa', 'Kolkata', 'Jaipur', 'Lucknow']

In [6]:
# creating empty lists to fetch required data

Airline_name = []       # The name of the airline
Departure_time = []     # The time when the journey starts from the source
Arrival_time = []       # Time of arrival at the destination
Duration = []           # Total duration taken by the flight to reach the destination from the source
Stops = []              # Total stops between the source and destination
Source = []             # The source from which the service begins
Destination = []        # The destination where the service ends
Meal = []               # Availability of meals in the flight
Price = []              # The price of th eflight ticket
Location = []           # The location of the flights

In [7]:
# scraping the data from the mentioned url

# fetching webelement of source
for x in loc_lst:
    for y in loc_lst:
        if x!=y:
            Source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(3)
            Source.clear()
            Source.send_keys(x)
            time.sleep(2)
            
            # fetching webelement of destination
            Destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            
            # search for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # scrolling the data 5000 times
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(4)
                
            # fetching web element of scroll to top button
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
            # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline_name.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    Departure_time.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching arrival time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # fteching duration of flights journey
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching availability of meal in flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching prices fo flight tickets
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass
            

In [8]:
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [9]:
# checking length of source and destination
len(Source), len(Destination)

(3455, 3455)

In [10]:
# checking lengths of all features
print(len(Price), len(Airline_name), len(Departure_time), len(Arrival_time), len(Duration), len(Stops), len(Meal))

3455 3355 3455 3455 3455 3455 3455


In [11]:
# creating a dataframe for scraped data
data = list(zip(Airline_name,Departure_time,Arrival_time,Duration,Source,Destination,Meal,Stops,Price))
df = pd.DataFrame(data, columns =["Airline",
                                  "Departure_time",  
                                  "Time_of_arrival",      
                                  "Duration",                                        
                                  "Source",        
                                  "Destination",
                                  "Meal_availability",
                                  "Number_of_stops",
                                  "Price"])
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Meal_availability,Number_of_stops,Price
0,Go First,08:45,10:55,2h 10m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
1,Go First,22:30,00:40,2h 10m,New Delhi,Mumbai,Emissions: 132 Kg CO2,Non Stop,"5,954"
2,Go First,14:20,16:35,2h 15m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
3,Go First,10:20,12:40,2h 20m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
4,Go First,08:25,14:00,5h 35m,New Delhi,Mumbai,eCash 250,1 Stop,"5,954"
...,...,...,...,...,...,...,...,...,...
3350,Air India,11:05,21:35,10h 30m,Lucknow,Hyderabad,Free Meal,1 Stop,"11,349"
3351,Air India,08:55,12:25,27h 30m,Lucknow,Hyderabad,Free Meal,2 Stop(s),"12,360"
3352,Air India,15:25,12:15,20h 50m,Lucknow,Hyderabad,No Meal Fare,2 Stop(s),"12,387"
3353,Air India,14:05,20:05,6h 00m,Lucknow,Goa,eCash 250,1 Stop,"7,463"


In [12]:
# saving the data into excel file
df.to_excel("Flight_Prices.xlsx")

So here I have successfully scraped the required data from the website yatra.com. The dataframe consists of 3355 rows and 9 columns. And I have saved the dataframe in excel format as "Flight_Prices"